In [14]:
import numpy as np
from scipy.stats import entropy
import torch
from seqCGAN.generator import Generator  # 假设你有一个定义好的 Discriminator 类
from seqCGAN.util import *
import json
import random
import math
import struct
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from scipy.stats import wasserstein_distance
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from dtaidistance import dtw
import os

In [15]:
# TOTAL_LEN = 114
# # NPRINT_REAL_WIDTH = 50*8
# NPRINT_REAL_WIDTH = 22*8
# LABEL_DICT = {'facebook': 0, 'skype': 1, 'aim': 2, 'email': 3, 'voipbuster': 4, 'hangouts': 5, 'youtube': 6, 'sftp': 7, 'icq': 8,  'ftps': 9, 'vimeo': 10, 'spotify': 11, 'netflix': 12, 'bittorrent': 13}
# LABEL_DICT = {'facebook': 0, 'skype': 1}
# LABEL_DICT = {'facebook': 0, 'skype': 1, 'email': 2, 'voipbuster': 3, 'hangouts': 4, 'youtube': 5, 'ftps': 6, 'vimeo': 7, 'spotify': 8, 'netflix': 9, 'bittorrent': 10}
# LABEL_DICT = {'facebook': 0, 'skype': 1, 'email': 2, 'voipbuster': 3, 'youtube': 4, 'ftps': 5, 'vimeo': 6, 'spotify': 7, 'netflix': 8, 'bittorrent': 9}
# LABEL_DICT = {'email': 0, 'youtube': 1, 'ftps': 2, 'vimeo': 3, 'spotify': 4, 'netflix': 5, 'bittorrent': 6}

# SEQ_DIM = 2
# MAX_PKT_LEN = 3001
# MAX_TIME = 10000
# MAX_PORT = 65535
# MAX_SEQ_LEN = 16
# WORD_VEC_SIZE = 8
# SEQ_DIM = WORD_VEC_SIZE * 2 + 1

label_dim = len(LABEL_DICT) 
# image_dim = (1, NPRINT_REAL_WIDTH, NPRINT_REAL_WIDTH)  # 生成单通道图像
noise_dim = 100  # 噪声维度
batch_size = 128
# source_name = './data/vpn_data_small.json'
# bins_name = './bins/bins_small_new.json'
data_folder = './data/' + DATASET + '/'
bins_file_name = './bins/bins_' + DATASET + '.json'
wordvec_file_name = './wordvec/word_vec_' + DATASET + '.json'
model_name = './save_seq/generator.pth'

with open(wordvec_file_name, 'r') as f:
    wv_dict = json.load(f)
    
wv = {}
for key, metrics in wv_dict.items():
    wv[key] = torch.tensor(metrics, dtype=torch.float32)

x_list = [wv_tensor.size(0) for wv_tensor in wv.values()]
# x_list = [MAX_TIME, MAX_PKT_LEN]

In [16]:
bins_data = {}
with open(bins_file_name, 'r') as f_bin:
    bins_data = json.load(f_bin)
    
# port_intervals = bins_data['port']['intervals']
# pkt_len_intervals = bins_data['packet_len']['intervals']
# time_intervals = bins_data['time']['intervals']
# for bins in bins_data['packet_len']:
#     pkt_len_intervals.append(bins['intervals'])
# time_intervals = []
# for bins in bins_data['time']:
#     time_intervals.append(bins['intervals'])

In [ ]:
# generator = Generator(label_dim, noise_dim, SEQ_DIM, MAX_SEQ_LEN, 'cpu')
generator = Generator(label_dim,SEQ_DIM,MAX_SEQ_LEN,x_list,'cpu')

# 加载模型权重
checkpoint = torch.load(model_name)  # 加载保存的权重字典
generator.load_state_dict(checkpoint)  # 将权重字典加载到模型中

# 切换到评估模式
generator.eval()

In [ ]:
for name, param in generator.named_parameters():
    print(f"Parameter name: {name}")
    print(param)

In [19]:
def get_real_data(file_name):
    
    data_dic = {}
    for filename in os.listdir(data_folder):
        with open(data_folder + filename, 'r') as f:
            if filename.split('.')[0] not in LABEL_DICT.keys():
                continue
            json_data = json.load(f)
            data_dic[filename.split('.')[0]] = []
            for item in json_data:
                meta_list = []
                for meta_attr in META_LIST:
                    meta_list.append(item[meta_attr])
                count = 0
                seq = []
                for pkt in item['series']:
                    attr_list = []
                    for sery_attr in SERY_LIST:
                        attr_list.append(pkt[sery_attr])
                    seq.append(attr_list + meta_list)
                    count += 1
                    if count >= MAX_SEQ_LEN:
                        break
                data_dic[filename.split('.')[0]].append(seq)
            # data_dic[filename.split('.')[0]] = json_data['data']
            # data += json_data
            # labels += [filename.split('.')[0]] * len(json_data)
    # with open(file_name,'r') as f:
    #     json_data = json.load(f)['data']
    # data_dic = {}
    
    # for item in json_data:
    #     label_str = item['labels'][0]
    #     if label_str not in data_dic:
    #         data_dic[label_str] = []

    #     packets_len = item['meta']['packets']
    #     im = bytes.fromhex(item['nprint'])

    #     line = im[0:TOTAL_LEN]
    #     tcp_dport = line[32:34]
    #     udp_dport = line[92:94]

    #     dport = bytearray(a | b for a, b in zip(tcp_dport, udp_dport))
    #     dport = int.from_bytes(dport, 'big')
    
    #     data_item = []
    #     for i in range(min(16,packets_len)):
    #         line = im[i*TOTAL_LEN:i*TOTAL_LEN+TOTAL_LEN]
    #         # time_h, time_l, pl = struct.unpack("IIh", line[:10])
    #         # time_l //= 1e4
    #         # time = time_h + time_l/100
    #         time_h,time_l, pkt_len = struct.unpack("IIh", line[:10])
    #         time_l //= 1e4
    #         time = time_h + time_l/100
    #         time = time_h % 1000 + time_l / 10
            
    #         data_item.append([time, pkt_len])
            
    #     data_dic[label_str].append(data_item)
        
    return data_dic


In [ ]:
real_datas = get_real_data(data_folder)
print(real_datas)

In [21]:
class SequenceDataset(Dataset):
    def __init__(self, datas, label_str):
        """
        :param sequences: 一个包含真实序列的列表，每个序列是一个 ndarray 或 list
        """
        self.datas = datas
        self.lengths = [len(seq) for seq in datas]  # 提取序列长度
        label_id = LABEL_DICT[label_str]
        self.label_one_hot = torch.zeros(label_dim).to('cpu') 
        self.label_one_hot[label_id] = 1

    def __len__(self):
        return len(self.datas)

    def __getitem__(self, idx):
        return self.lengths[idx],self.label_one_hot 

In [22]:
def get_fake_data(real_data, label_str):
    dataset = SequenceDataset(real_data,label_str)
    dataloader = DataLoader(dataset, batch_size=128, shuffle=False)
    generated_sequences = []
    with torch.no_grad():
        for lengths, labels in dataloader:
            lengths = lengths.to(torch.device("cpu"))  # 确保在同一个设备上
            labels = labels.to(torch.device("cpu"))
            # print(lengths.shape)
            # print(labels.shape)
            batch_size = lengths.size(0)
            # 生成随机噪声向量
            # noise = torch.randn(len(lengths), noise_dim)
            # 输入生成器生成数据
            fake_data = generator.sample(batch_size,labels,lengths)
            # 将生成结果按序列长度截断
            for i, length in enumerate(lengths):
                generated_sequences.append(fake_data[i, :length].cpu().tolist())
       
    final_seqs = []         
    for seq in generated_sequences:
        f_seq = []
        for i in range(len(seq)):
            pkt = []
            for j,attr_id in enumerate(seq[i]): 
                if j == 0:
                    attr = round(random.uniform(bins_data[SEQ_LIST[j]]['intervals'][attr_id][0], bins_data[SEQ_LIST[j]]['intervals'][attr_id][1]),2)
                else:
                    attr = round(random.uniform(bins_data[SEQ_LIST[j]]['intervals'][attr_id][0], bins_data[SEQ_LIST[j]]['intervals'][attr_id][1]))
                pkt.append(attr)
            # [time,pkt_len] = seq[i]       
            # # time_id = math.floor((time + 1)/2*len(time_intervals[i]))
            # # pkt_len_id = math.floor((pkt_len + 1)/2*len(pkt_len_intervals[i]))
            # # dport_id = math.floor((dport + 1)/2*len(port_intervals))
        
            # time = round(random.uniform(time_intervals[time][0], time_intervals[time][1]),2)
            # pkt_len = round(random.uniform(pkt_len_intervals[pkt_len][0], pkt_len_intervals[pkt_len][1]))
            # dport = round(random.uniform(port_intervals[dport_id][0],port_intervals[dport_id][1]))
            # time = time
            # pkt_len = pkt_len
            # f_seq.append([time,pkt_len])
            f_seq.append(pkt)
        final_seqs.append(f_seq)
    return final_seqs

In [23]:
def pad(sequence, target_length, pad_value=np.nan):
    seq_len = len(sequence)
    if seq_len < target_length:
        padding = [[pad_value] * len(sequence[0])] * (target_length - seq_len)
        return sequence + padding  # 填充
    return sequence

In [ ]:
fake_datas = {}
for label, data in real_datas.items():
    # fake_data = get_fake_data_wv(data,label,wv)
    fake_data = get_fake_data(data,label)
    fake_datas[label] = fake_data
    # for i in range(1):
    #     fake_data = get_fake_data(data,label)
    #     fake_datas[label] += fake_data
    # print(label)
    # print(data)
    # print(fake_data)
    # real_grouped = group_by_length(data)
    # fake_grouped = group_by_length(fake_data)
    # overall_js_divergence = calculate_divergence(real_grouped, fake_grouped)
    # print(f"Overall JS Divergence {label}: {overall_js_divergence}")

In [ ]:
for label in LABEL_DICT.keys():
    real_data = real_datas[label]
    fake_data = fake_datas[label]
    real_sequences = np.array([pad(seq, MAX_SEQ_LEN) for seq in real_data])         # Shape: (num_samples, seq_len, num_dims)
    generated_sequences = np.array([pad(seq, MAX_SEQ_LEN) for seq in fake_data]) 

    num_samples, seq_len, num_dims = real_sequences.shape
    # print(real_sequences.shape)

    for dim in range(num_dims):
        
        percentiles = [10, 90]
        quantiles = np.nanpercentile(real_sequences[:,:,dim], percentiles, axis=0)
        
        plt.figure(figsize=(10, 6))
        
        if dim == 0:
            plt.axis([0, 16, 0, 3000])
        elif dim == 1:
            plt.axis([0, 16, -1500, 1500])
        elif dim == 2 or dim == 3:
            plt.axis([0, 16, 0, 255])
        elif dim == 4 or dim == 5:
            plt.axis([0, 16, 0, 65535])
        else:
            plt.axis([0, 16, 0, 4294967295])

        # 绘制真实数据序列（统一颜色）
        for i in range(len(real_sequences)):
            plt.plot(real_sequences[i, :, dim], color='blue', alpha=1.0, label="Real" if i == 0 else "")  # 使用蓝色，透明度为 0.6
    
        # 绘制生成数据序列（统一颜色）
        for i in range(len(generated_sequences)):
            plt.plot(generated_sequences[i, :, dim], color='red', alpha=0.5, linestyle='--', label="Generated" if i == 0 else "")  # 使用红色，透明度为 0.6
            
        # for q in quantiles:
        #     plt.plot(q, color='green', linestyle='--', linewidth=2.5, label='Real Data Quantiles' if 'Real Data Quantiles' not in plt.gca().get_legend_handles_labels()[1] else "")
    
        plt.title(f"Dimension {dim + 1} Comparison of {label}")
        plt.xlabel("Time Step")
        plt.ylabel("Value")
        plt.legend()
        plt.grid(True)
        plt.show()
        

In [ ]:
import ot  # optimal transport library

def hamming_distance_matrix(X, Y):
    X = np.array(X)
    Y = np.array(Y)
    batch_size = len(X)
    cost_matrix = np.zeros((batch_size, batch_size))
    for i in range(batch_size):
        for j in range(batch_size):
            cost_matrix[i, j] = np.sum(X[i] != Y[j])
    return cost_matrix


X = np.array([
    [[0, 1], [1, 0]],  # 样本 1
    [[2, 3], [3, 2]]   # 样本 2
])
Y1 = np.array([
    [[2, 3], [3, 2]],  # 样本 1
    [[0, 1], [1, 0]]   # 样本 2
])
Y2 = np.array([
    [[100, 101], [101, 100]],  # 样本 1
    [[-98, -97], [-97, -98]]   # 样本 2
])

# 计算 X 和 Y 之间所有样本对的距离矩阵
# cost_matrix_Y1 = np.linalg.norm(X[:, None, :, :] - Y1[None, :, :, :], axis=(-2, -1))
# cost_matrix_Y2 = np.linalg.norm(X[:, None, :, :] - Y2[None, :, :, :], axis=(-2, -1))
cost_matrix_Y1 = hamming_distance_matrix(X, Y1)
cost_matrix_Y2 = hamming_distance_matrix(X, Y2)

# 计算最优传输距离
ot_distance_Y1 = ot.emd2([], [], cost_matrix_Y1)  # Optimal Transport Distance
ot_distance_Y2 = ot.emd2([], [], cost_matrix_Y2)

print("OT 距离 (X vs Y1, 预期相同):", ot_distance_Y1)  # 应该接近 0
print("OT 距离 (X vs Y2, 预期不同):", ot_distance_Y2)  # 应该很大


In [ ]:
# optimal transport distance
for label in LABEL_DICT.keys():
    real_data = real_datas[label]
    fake_data = fake_datas[label]
    real_sequences = np.array([pad(seq, MAX_SEQ_LEN) for seq in real_data])         # Shape: (num_samples, seq_len, num_dims)
    generated_sequences = np.array([pad(seq, MAX_SEQ_LEN) for seq in fake_data]) 

    num_samples, seq_len, num_dims = real_sequences.shape
    # print(real_sequences.shape)

    # for dim in range(num_dims):
    X = real_sequences
    Y = generated_sequences
        
        # Z = np.random.rand(Y.shape[0], Y.shape[1])  # 随机生成一个矩阵
        
    X_filled = np.nan_to_num(X, nan=10000)  # 或者 nan=0，如果认为0不影响
    Y_filled = np.nan_to_num(Y, nan=10000)
    
    # X_filled = X_filled / 1500
    # Y_filled = Y_filled / 1500
        
    # cost_matrix = np.linalg.norm(np.cumsum(X, axis=0)[:, None, :] - np.cumsum(Y, axis=0)[None, :, :], axis=-1)
    cost_matrix = np.linalg.norm(X_filled[:, None, :, :] - Y_filled[None, :, :, :], axis=(-2, -1))
        
    # print(cost_matrix.shape)
    ot_distance = ot.emd2([], [], cost_matrix) 
        
    print(f"OT 距离 {label} {dim}:", ot_distance)
        
        # percentiles = [10, 90]
        # quantiles = np.nanpercentile(real_sequences[:,:,dim], percentiles, axis=0)
        
        # plt.figure(figsize=(10, 6))
        
        # if dim == 0:
        #     plt.axis([0, 16, 0, 3000])
        # else:
        #     plt.axis([0, 16, -1500, 1500])

        # # 绘制真实数据序列（统一颜色）
        # for i in range(len(real_sequences)):
        #     plt.plot(real_sequences[i, :, dim], color='blue', alpha=1.0, label="Real" if i == 0 else "")  # 使用蓝色，透明度为 0.6
    
        # # 绘制生成数据序列（统一颜色）
        # for i in range(len(generated_sequences)):
        #     plt.plot(generated_sequences[i, :, dim], color='red', alpha=0.5, linestyle='--', label="Generated" if i == 0 else "")  # 使用红色，透明度为 0.6